In [1]:
from funcs import *
import os
from collections import defaultdict

In [2]:
str1 = r"""\begin{table}[!htb]
\centering
\caption{Параметры задачи для экспериментального исследования}\label{task_params}
\begin{tabular}{|c|c|c|c|}
\hline
$\delta$ & Всего кандидатов & Ровно $\delta$ & Всего грязных \\ \hline"""

str2 = r'''\begin{table}[!htb]
\centering
\caption{Параметры задачи для экспериментального исследования}\label{task_params}
\begin{tabular}{|c|c|c|c|}
\hline
$\delta$ & Грязных после очищения & Успешно очищенных & Всего грязных \\ \hline'''

table_end = r'''\end{tabular}
\end{table}
'''

line = r'${0}$ & ${1}$ & ${2}$ & ${3}$ \\ \hline'

In [3]:
def make_table(s, table_head):
    with open(s, "r", encoding="utf-8") as f:
        raw_text = f.read()

    data = {}

    for line in raw_text.strip().split('\n'):
        key_str, val_str = line.split(':', 1)
        key = int(key_str.strip())
        val_dict = ast.literal_eval(val_str.strip())
        data[key] = val_dict

    x = sorted(data.keys())
    res = []
    total_counts = []
    avg_values = []
    count_1s = []
    delta_s = []

    res_table = copy(table_head)
    for k in x:
        inner = data[k]
        total = sum([k * inner_k * inner[inner_k] for inner_k in inner]) + 0.0

        exp_c = sum(inner.values())
        res.append(k * exp_c)

        weighted_avg = sum([(k * inner_k * inner[inner_k]) for inner_k in inner]) / (exp_c)

        count_1 = inner.get(1.0, 0) * k

        delta_s.append(k)
        total_counts.append(total)
        avg_values.append(weighted_avg)
        count_1s.append(count_1)

        new_line = (r'${0}$ & ${1}$ & ${2}$ & ${3}$ \\ \hline').format(k, int(total), count_1, k * exp_c)
        res_table += ("\n"+new_line)

    res_table += ("\n"+table_end)

    print(res_table)


In [4]:
folder = "/Users/trcmkr/Downloads/RM(2_7)_L20_R46_E100-1"

make_table(folder+"/candidates_freq.txt", str1)
make_table(folder+"/distillation_freq.txt", str2)
make_table(folder+"/clearing_freq.txt", str2)

\begin{table}[!htb]
\centering
\caption{Параметры задачи для экспериментального исследования}\label{task_params}
\begin{tabular}{|c|c|c|c|}
\hline
$\delta$ & Всего кандидатов & Ровно $\delta$ & Всего грязных \\ \hline
$20$ & $2000$ & $2000$ & $2000$ \\ \hline
$21$ & $2100$ & $2100$ & $2100$ \\ \hline
$22$ & $2200$ & $2200$ & $2200$ \\ \hline
$23$ & $2300$ & $2300$ & $2300$ \\ \hline
$24$ & $2400$ & $2400$ & $2400$ \\ \hline
$25$ & $2500$ & $2500$ & $2500$ \\ \hline
$26$ & $2600$ & $2600$ & $2600$ \\ \hline
$27$ & $2700$ & $2700$ & $2700$ \\ \hline
$28$ & $2800$ & $2800$ & $2800$ \\ \hline
$29$ & $2900$ & $2900$ & $2900$ \\ \hline
$30$ & $3000$ & $3000$ & $3000$ \\ \hline
$31$ & $3100$ & $3100$ & $3100$ \\ \hline
$32$ & $3200$ & $3200$ & $3200$ \\ \hline
$33$ & $3300$ & $3300$ & $3300$ \\ \hline
$34$ & $3400$ & $3400$ & $3400$ \\ \hline
$35$ & $3500$ & $3500$ & $3500$ \\ \hline
$36$ & $3600$ & $3600$ & $3600$ \\ \hline
$37$ & $3700$ & $3700$ & $3700$ \\ \hline
$38$ & $3800$ & $3800$ & $

In [5]:
def fill_freq(folder, expc, left):
    tmp = []
    with open(folder + "/candidates.json", "r") as f:
        tmp = json.load(f)

    candidates = []
    for x in tmp:
        matr = load(f'{folder}/matrices/{x[1]}.sobj')
        candidates.append([x[0], matr, x[2], x[3]])

    freq_map = defaultdict(lambda: defaultdict(int))

    for x in candidates:
        freq_map[x[0]][len(x[2]) / max(len(x[3]), 1)] += 1

    find_noised_candidates_res_freq = {k: dict(v) for k, v in freq_map.items()}

    with open(folder + "/candidates_freq.txt", "w") as f:
        for key in find_noised_candidates_res_freq:
            print(key, ":", find_noised_candidates_res_freq[key], end='\n', file=f)

    # -----

    tmp = []
    with open(folder + "/clearing.json", "r") as f:
        tmp = json.load(f)

    freq_map = defaultdict(lambda: defaultdict(int))

    for i in range(len(tmp)):
        ind = tmp[i][0]
        freq_map[int(ind / expc + left)][len(set(tmp[i][1]) & set(candidates[ind][3])) / max(len(candidates[ind]), 1)] += 1

    get_noised_freq = {k: dict(v) for k, v in freq_map.items()}

    with open(folder + "/clearing_freq.txt", "w") as f:
        for key in get_noised_freq:
            print(key, ":", get_noised_freq[key], end='\n', file=f)

    # -----

    tmp = []
    with open(folder + "/distillation.json", "r") as f:
        tmp = json.load(f)

    req_map = defaultdict(lambda: defaultdict(int))

    for i in range(len(tmp)):
        req_map[int(i / expc + left)][len(set(tmp[f'{i}']) & set(candidates[i][3])) / max(len(candidates[i][3]), 1)] += 1

    distillation_freq = {k: dict(v) for k, v in req_map.items()}

    with open(folder + "/distillation_freq.txt", "w") as f:
        for key in distillation_freq:
            print(key, ":", distillation_freq[key], end='\n', file=f)

In [6]:
fill_freq("/Users/trcmkr/Downloads/RM(2_7)_L20_R46_E100-1", 100, 20)